In [1]:
import ee 
from ee_jupyter.ipyleaflet import Map
import ipyleaflet as ipl

ee.Initialize()

上面的单元格是普通的初始化, 下面开始导入数据集, 观察是否能用

In [10]:
modis=ee.ImageCollection("MODIS/061/MYD08_M3")
start='2019-01-01'
end='2019-02-01'
bands=modis.first().bandNames().getInfo()
for band in bands:
    print(band)

Solar_Zenith_Mean_Mean
Solar_Zenith_Std_Deviation_Mean
Solar_Azimuth_Mean_Mean
Solar_Azimuth_Std_Deviation_Mean
Sensor_Zenith_Mean_Mean
Sensor_Zenith_Std_Deviation_Mean
Sensor_Azimuth_Mean_Mean
Sensor_Azimuth_Std_Deviation_Mean
Aerosol_Scattering_Angle_Mean_Mean
Aerosol_Scattering_Angle_Std_Deviation_Mean
Aerosol_Avg_Cloud_Distance_Land_Ocean_Mean_Mean
Aerosol_Avg_Cloud_Distance_Land_Ocean_Std_Deviation_Mean
Aerosol_Optical_Depth_Land_Ocean_Mean_Mean
Aerosol_Optical_Depth_Land_Ocean_Std_Deviation_Mean
Aerosol_Optical_Depth_Land_Mean_Mean_470
Aerosol_Optical_Depth_Land_Mean_Mean_550
Aerosol_Optical_Depth_Land_Mean_Mean_660
Aerosol_Optical_Depth_Land_Std_Deviation_Mean_470
Aerosol_Optical_Depth_Land_Std_Deviation_Mean_550
Aerosol_Optical_Depth_Land_Std_Deviation_Mean_660
Aerosol_Optical_Depth_Land_QA_Mean_Mean_470
Aerosol_Optical_Depth_Land_QA_Mean_Mean_550
Aerosol_Optical_Depth_Land_QA_Mean_Mean_660
Aerosol_Optical_Depth_Land_QA_Std_Deviation_Mean_470
Aerosol_Optical_Depth_Land_QA_Std_D

先试试'光学厚度'这一个对象

In [14]:
cloud=modis.filterDate(start,end).select('Cloud_Optical_Thickness_Liquid_Mean_Mean')
map1=Map(center=[40,-100],zoom=4,basemap=ipl.basemaps.Esri.WorldImagery)
map1.addLayer(cloud.first(),{'min':0,'max':5000},'cloud')
map1

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

试一试新导入的china.json文件, 另外发现了宝藏网站https://geo.datav.aliyun.com/

In [24]:
import json

with open(r'..\shp\china.json', 'r') as file:
    json_data = json.load(file)

china = ee.FeatureCollection(json_data)
boundary = ee.Image().paint(china, 0, 1)
map1.addLayer(boundary, {'palette': 'gray'}, 'boundary')


<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Windows\Temp\ipykernel_16016\887847855.py:3: SyntaxWarning: invalid escape sequence '\s'
  with open('..\shp\china.json', 'r') as file:


让我调整一些可视化的色彩

In [25]:
clouds_china = cloud.map(lambda image: image.clip(china))
map2=Map(center=[40,-100],zoom=4,basemap=ipl.basemaps.Esri.WorldImagery)
map2.addLayer(clouds_china.first(),{'min':0,'max':5000,'palette':['blue','skyblue','white']},'cloud')
map2.addLayer(boundary, {'palette': 'gray'}, 'boundary')
map2

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

所以现在会使用china的clip操作, 问题在于:精度太低了,让我重拾S2

In [30]:
region = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])  # 示例区域

# 加载Sentinel-2影像集
s2 =ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY").filterDate(start, end)

# 定义一个函数来提取云量百分比
def get_cloudy_percentage(image):
    return image.select('probability')

# 应用函数并计算平均云量
cloud_percentage = s2.map(get_cloudy_percentage)
mean_cloud_percentage = cloud_percentage.mean().clip(china)

# 创建地图并添加平均云量图层
map3 = Map(center=[39.916, 116.397], zoom=6,basemap= ipl.basemaps.Esri.WorldImagery)  # 地图中心设置在北京
map3.addLayer(mean_cloud_percentage, {'min':0,'max':100,'palette':['blue','skyblue','white']}, 'Mean Cloud Percentage')

# 显示地图
map3

Map(center=[39.916, 116.397], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

仔细看看上面的图像, 发现对于积水区域比如农田, 很容易出现云的误判, 因此我们还是选用MODIS

那么接下来的一个问题时MODIS的精度稍微有点低, 让我们试试插值

In [37]:
clouds_china_interpolated = clouds_china.map(lambda image: image.resample('bicubic').reproject('EPSG:4326', None, 500))
map4=Map(center=[40,117],zoom=4,basemap=ipl.basemaps.Esri.WorldImagery)
map4.addLayer(clouds_china_interpolated.first(),{'min':0,'max':5000,'palette':['blue','skyblue','white']},'cloud')
map4.addLayer(boundary, {'palette': 'gray'}, 'boundary')
map4

Map(center=[40, 117], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

找到一个新的数据集, 看看效果

In [42]:
another_s2=ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CLOUD").filterDate(start,end)
china_clouds_s5p=another_s2.select('cloud_optical_depth').mean().clip(china)
map5=Map(center=[40,117],zoom=4,basemap=ipl.basemaps.Esri.WorldImagery)
map5.addLayer(china_clouds_s5p,{'min':1,'max':50,'palette':['blue','skyblue','white']},'cloud')
map5.addLayer(boundary, {'palette': 'gray'}, 'boundary')
map5

Map(center=[40, 117], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [40]:
another_s2.size().getInfo()

348

从上面也可以看到另外一个数据集时间精度貌似挺高的?空间分辨率也很高

那么关于云影像集的选取这一方面就不应该有什么问题了